In [1]:
import numpy as np
import torch
import random
seed=0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# load data

In [2]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [3]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Normal\1.jpg,1
1,../dataset/wrist_xray/Normal\30.jpg,1
2,../dataset/wrist_xray/Fracture\47 Male (A View...,0
3,../dataset/wrist_xray/Normal\20.jpg,1
4,../dataset/wrist_xray/Normal\52.jpg,1


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [4]:
import sys
sys.path.append("../src") 

In [5]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel1,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

In [7]:
print('MyNet parameters :' ,count_parameters(mymodel1()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 1.158232
SqueezeNet parameters : 1.234344
ShuffleNet parameters : 1.36636
MnastNet parameters : 2.218224
MobileNet parameters : 3.504296
DenseNet parameters : 7.972584


In [8]:
epoch=32
batchsize=8
lr=0.001

dim='gray' for gray scale images  
skip_tuning=False, no grid search applied for hyper-parameter tuning,  
aug=1-> pytorch augmentation  
aug=0-> no augmentation  
aug=2-> albumentation augmentation    
aug=3-> rand augmentation

# OurNet

In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.854   0.834     0.840
Class 1       0.786   0.822     0.796
macro         0.820   0.828     0.818
weighted      0.832   0.824     0.824
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.876   0.860     0.868
Class 1       0.820   0.844     0.830
macro         0.850   0.852     0.850
weighted      0.858   0.856     0.854
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.862   0.858     0.856
Class 1       0.806   0.830     0.810
macro         0.834   0.844     0.832
weighted      0.846   0.838     0.838
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.856   0.832     0.842
Class 1       0.784   0.826     0.798
macro         0.820   0.828     0.820
weighted      0.836   0.824     0.826
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.72,0.87,0.87,0.77
1,0.82,0.85,0.79,0.90
2,0.90,0.90,0.90,0.79
3,0.84,0.79,0.79,0.79
4,0.84,0.87,0.84,0.87
mean,0.824,0.856,0.838,0.824


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.614   0.578     0.592
Class 1       0.486   0.510     0.486
macro         0.548   0.542     0.540
weighted      0.568   0.564     0.560
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.600   0.568     0.582
Class 1       0.434   0.460     0.442
macro         0.516   0.512     0.512
weighted      0.538   0.536     0.536
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.588   0.574     0.582
Class 1       0.434   0.440     0.432
macro         0.508   0.504     0.504
weighted      0.536   0.524     0.526
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.682   0.690     0.668
Class 1       0.514   0.494     0.462
macro         0.600   0.590     0.566
weighted      0.618   0.606     0.582
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.54,0.74,0.77,0.79
1,0.54,0.38,0.41,0.46
2,0.69,0.64,0.49,0.64
3,0.50,0.47,0.50,0.53
4,0.55,0.45,0.45,0.61
mean,0.564,0.536,0.524,0.606


# ShuffleNet

In [11]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.562   0.622     0.574
Class 1       0.436   0.372     0.378
macro         0.500   0.498     0.476
weighted      0.526   0.518     0.498
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.614   0.672     0.622
Class 1       0.468   0.422     0.418
macro         0.542   0.546     0.520
weighted      0.566   0.542     0.532
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.638   0.668     0.642
Class 1       0.494   0.482     0.470
macro         0.566   0.574     0.554
weighted      0.588   0.576     0.566
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.596   0.712     0.638
Class 1       0.464   0.348     0.378
macro         0.530   0.530     0.506
weighted      0.554   0.544     0.528
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.62,0.46,0.44,0.51
1,0.56,0.56,0.56,0.54
2,0.51,0.56,0.67,0.59
3,0.45,0.58,0.58,0.53
4,0.45,0.55,0.63,0.55
mean,0.518,0.542,0.576,0.544


# Mnasnet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.552   0.584     0.558
Class 1       0.376   0.352     0.348
macro         0.464   0.470     0.454
weighted      0.490   0.478     0.472
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.558   0.634     0.592
Class 1       0.402   0.330     0.360
macro         0.482   0.482     0.476
weighted      0.500   0.512     0.502
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.562   0.570     0.562
Class 1       0.356   0.376     0.364
macro         0.458   0.472     0.462
weighted      0.482   0.488     0.480
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.554   0.600     0.566
Class 1       0.424   0.370     0.380
macro         0.488   0.486     0.474
weighted      0.510   0.502     0.492
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.38,0.33,0.31,0.54
1,0.44,0.51,0.54,0.38
2,0.46,0.69,0.56,0.54
3,0.58,0.53,0.53,0.55
4,0.53,0.50,0.50,0.50
mean,0.478,0.512,0.488,0.502


# MobileNet

In [13]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.552   0.630     0.568
Class 1       0.434   0.320     0.320
macro         0.492   0.474     0.446
weighted      0.512   0.480     0.462
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.588   0.720     0.648
Class 1       0.462   0.326     0.382
macro         0.526   0.526     0.514
weighted      0.542   0.554     0.538
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.606   0.692     0.642
Class 1       0.464   0.394     0.412
macro         0.538   0.542     0.528
weighted      0.560   0.562     0.546
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.638   0.646     0.638
Class 1       0.496   0.490     0.488
macro         0.570   0.570     0.564
weighted      0.590   0.586     0.582
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.56,0.49,0.44,0.62
1,0.38,0.56,0.56,0.54
2,0.46,0.72,0.62,0.64
3,0.42,0.45,0.45,0.50
4,0.58,0.55,0.74,0.63
mean,0.48,0.554,0.562,0.586


# DenseNet121

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.638   0.574     0.602
Class 1       0.470   0.528     0.494
macro         0.552   0.552     0.548
weighted      0.578   0.564     0.568
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.714   0.694     0.698
Class 1       0.604   0.634     0.610
macro         0.656   0.666     0.654
weighted      0.680   0.664     0.666
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.712   0.678     0.688
Class 1       0.586   0.628     0.596
macro         0.650   0.654     0.644
weighted      0.670   0.654     0.656
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.728   0.636     0.676
Class 1       0.568   0.662     0.604
macro         0.644   0.650     0.640
weighted      0.670   0.648     0.654
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.59,0.62,0.64,0.72
1,0.46,0.62,0.56,0.56
2,0.54,0.72,0.67,0.59
3,0.55,0.68,0.66,0.66
4,0.68,0.68,0.74,0.71
mean,0.564,0.664,0.654,0.648
